In [21]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re 
from urllib.request import urlopen
import pandas as pd
import numpy as np

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait

In [22]:
def insta_searching(word):  #word라는 매개변수를 받는 insta_searching 이라는 함수 생성
    url = 'https://www.instagram.com/explore/tags/' + word
    return url


In [23]:
import time 
def select_first(driver):
    first = wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div[1]/div[2]')))
   
    first.click()
    time.sleep(3)

In [25]:
import re 
def get_content(driver,name):
    # 현재 페이지의 HTML 정보 가져오기
    time.sleep(np.random.randint(0.3,1))
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    
    name = name
    # 본문 내용 가져오기
    try: #여러 태그중 첫번째([0]) 태그를 선택  
        content = soup.select('div.C4VMK > span')[0].text 
        #첫 게시글 본문 내용이 <div class="C4VMK"> 임을 알 수 있다.
                                #태그명이 div, class명이 C4VMK인 태그 아래에 있는 span 태그를 모두 선택.
    except:
        content = ' ' 
    # 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content) # content 변수의 본문 내용 중 #으로 시작하며, #뒤에 연속된 문자(공백이나 #, \ 기호가 아닌 경우)를 모두 찾아 tags 변수에 저장
    # 작성 일자 가져오기
    time.sleep(0.4)
    try:
        date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10] #앞에서부터 10자리 글자
    except:
        date = ''
    # 좋아요 수 가져오기
    try : 
        like = driver.find_elements_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/section[2]/div/div/a') #안되서 수정한코드(정상작동)*
       #/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/section[2]/div/div
        likes = like[0].text #print 했을 때 좋아요 수 그대로 크롤링해줌 * 
    except :
        likes = 0 # 단 '조회'로 넘어가니까 작동 멈춤(except 조건 넣어서 like가 아닐때 0으로 뜨게 함)

    # 위치 정보 가져오기
    try:
        place = soup.select('div.JF9hh')[0].text
    except:
        place = ''
    time.sleep(0.3)
    try: 
        link = soup.select_one('a.sqdOP.yWX7d').get('href')
    except:
        time.sleep(3)
        link =''
    
    # 수집한 정보 저장하기
    data = {'name' : name,
            'content':content,
            'date' :date,
            'like' : likes,
            'place' : place, 
            'tags' : tags,
            'link' : 'https://www.instagram.com' +link}
    
    return data 

In [34]:
def move_next(driver):
    try:
        right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow') 
        right.click()
        
    except:
        pass

    time.sleep(3)

In [35]:
def get_result(name, mail, pw, number):
    
    driver = webdriver.Chrome('C:/chromedriver.exe')
    driver.maximize_window() #최대 사이즈로 조절
    url_log = 'https://www.instagram.com/accounts/login/'
    
    # 로그인 하기
    driver.get(url_log)
    time.sleep(4)

    driver.find_element_by_xpath('.block-page_button2').click() # 카이츠키퍼
    time.sleep(2.5)

    driver.find_element_by_name("username").send_keys(mail)
    driver.find_element_by_name("password").send_keys(pw)
    driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()
    time.sleep(5)
    

    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click() # 팝업 1
    time.sleep(2)

    driver.find_element_by_css_selector('.aOOlW.HoLwm').click() #팝업 2
    time.sleep(2) 

    # 검색페이지 접속하기
    url = insta_searching(name)
    driver.get(url)
    time.sleep(3) 

    # 첫번째 게시글 열기
    select_first(driver) 

    global results
    results = [] 

    #여러 게시물 크롤링하기
    for i in range(number):
        data = get_content(driver,name) #게시물 정보 가져오기
 
        results.append(data)
        print(i,'/',number)
        time.sleep(np.random.randint(1,3))
        move_next(driver)
        time.sleep(4.5)
        
    driver.quit()
    
    df_total = pd.DataFrame(results)
    return df_total

In [ ]:
get_result('태화강납량축제', 'id', 'password' , 500) # id/pwd 기입 ,500 : 크롤링할 포스트 갯수

In [19]:
df = pd.DataFrame(results)
df

,name,content,date,like,place,tags,link
0,태화강납량축제,혜띠랑 #태화강납량축제 #밤바다,2020-08-17,좋아요 155개,"Ulsan, South Korea","[#태화강납량축제, #밤바다]",https://www.instagram.com/q_0yi/
1,태화강납량축제,방구석퇴마사 3인 출동👹,2020-08-15,좋아요 49개,태화강대공원,[],https://www.instagram.com/ch_ye_ch/
2,태화강납량축제,더워 💦#태화강납량축제 #납량축제,2020-08-15,좋아요 304개,대나무숲,"[#태화강납량축제, #납량축제]",https://www.instagram.com/hoho_ho_o/
3,태화강납량축제,"무서워요👻작년에 이어 올해도 우연히, #태화강납량축제",2020-08-14,좋아요 126개,태화강대공원,[#태화강납량축제],https://www.instagram.com/daintime/
4,태화강납량축제,오싹오싹 #태화강_대숲_납량축제.올해는 8월1일~8월5일까지 열림호러트레킹 입장료 ...,2018-07-09,"좋아요 1,283개",,"[#태화강_대숲_납량축제.올해는, #울산플래닛, #울플축제, #울산축제, #여름축제...",https://www.instagram.com/ulsanplanet/
...,...,...,...,...,...,...,...
258,태화강납량축제,#instasize #울산 #울산태화강대공원 #십리대밭 #태화강납량축제 #태화강대숲...,2015-08-15,좋아요 26개,,"[#instasize, #울산, #울산태화강대공원, #십리대밭, #태화강납량축제, ...",https://www.instagram.com/hyojeongj3/
259,태화강납량축제,#세모녀 #태화강납량축제 행차중... #울산#납량축제#family #wife #da...,2015-08-15,좋아요 25개,,"[#세모녀, #태화강납량축제, #울산, #납량축제, #family, #wife, #...",https://www.instagram.com/turbo8579/
260,태화강납량축제,하아😥 지오디콘서트도 아닌데 무려 4시간을 기다리고.......또이가 무척 보고싶어...,2015-08-14,좋아요 23개,,[],https://www.instagram.com/someberry_/
261,태화강납량축제,#먹스타그램 #타코야끼 #문어빵 #태화강납량축제,2014-08-14,좋아요 8개,,"[#먹스타그램, #타코야끼, #문어빵, #태화강납량축제]",https://www.instagram.com/jaw_hyel/


In [20]:
df.to_csv('./태화강납량축제_0927.csv',encoding='utf-8-sig') 